In [1]:
import os

import PIL
from PIL import Image
from PIL.ImageDraw import Draw

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from scipy.ndimage import affine_transform
from tensorflow.keras import backend as K

import pandas as pd
from tqdm import tqdm

In [2]:
# !wget https://www.kaggleusercontent.com/kf/4346884/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..DyxYyheDh_RA5kd4_MHPwQ.875NZZPJbHPYKhkWxQjO1tIJ01qGMAJ694LuYv3CyQNrCrpFKPRXXjhvOt7-Aa6SxPWVNeo5F1_8K25uGevhDpq4xSvoeoqwNt1t0iMTglPBXCZh5cua_j4eZ2ezs9MnAKzQBeyxDxj8Z6cYpxnat56CkDUIIbjnJZL0o47Afgdk9nmIMZtlgfDJ-OH4XZ1Pv-Eg6wyw9ngGBjIVCjnxCxSHS2lYalz-IsIP5wRIjOxVMrUMDgR_AGgkCIvOgkHpMDXTpoPHJ53czO3zkMQ86GBIQAQcoZcHQzpkdE-gEaeDycqK5B_gH865TtbLK64tw0Rr2_F7u8uikrKgqVHMU8tflZbyzUbWconcEZ24dQ7ojWkuCs2mvdiIyoWLzaPCsE_pMoLAo1i0ytrEWyGHemb9kTnaVDRIIlktevTAefNf0uPzwfQE-U9MwzBcpJtKQ0magKgLg8Xu6mV_YmcU0zFx4Wl51cU9doB-9kh1L9ot6y9sIKAD733cqmiCvhblmK_cwU5hAt7cW28BiaVrtoIkJuAkvrkhvmDIYhHFX4klj9Yv4TAKq0BZ8rmmXuc403MRPLNGqSLYz5Rv-jltXxnQMYIt2kCZmHGklbJU6OUyY2WXhCzZ1yZB1F-mtvd2vwwb_P6HOk8zLCOx5ODy5DMcFA5s5-eb2Qpo9ADpUq8.fJT6QVF86y6-GG6dz0fkzA/cropping.model

In [3]:
MODEL_BASE = '../input/bounding-box-model'
DATA = '../WC_input/data'

In [4]:
model = load_model('cropping.model')

In [5]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d_18 (Conv2D)              (None, 128, 128, 64) 5248        input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_19 (Conv2D)              (None, 128, 128, 64) 36928       conv2d_18[0][0]                  
__________________________________________________________________________________________________
batch_normalization_7 (BatchNor (None, 128, 128, 64) 256         conv2d_19[0][0]                  
____________________________________________________________________________________________

In [6]:
data_paths = [img for img in os.listdir(DATA)]

In [7]:

# Define useful constants
img_shape  = (128, 128, 1)
anisotropy = 2.15

def center_transform(affine, input_shape):
    hi, wi = float(input_shape[0]), float(input_shape[1])
    ho, wo = float(img_shape[0]), float(img_shape[1])
    top, left, bottom, right = 0, 0, hi, wi
    if wi/hi/anisotropy < wo/ho: # input image too narrow, extend width
        w     = hi*wo/ho*anisotropy
        left  = (wi-w)/2
        right = left + w
    else: # input image too wide, extend height
        h      = wi*ho/wo/anisotropy
        top    = (hi-h)/2
        bottom = top + h
    center_matrix   = np.array([[1, 0, -ho/2], [0, 1, -wo/2], [0, 0, 1]])
    scale_matrix    = np.array([[(bottom - top)/ho, 0, 0], [0, (right - left)/wo, 0], [0, 0, 1]])
    decenter_matrix = np.array([[1, 0, hi/2], [0, 1, wi/2], [0, 0, 1]])
    return np.dot(np.dot(decenter_matrix, scale_matrix), np.dot(affine, center_matrix))

# Apply an affine transformation to an image represented as a numpy array.
def transform_img(x, affine):
    matrix   = affine[:2,:2]
    offset   = affine[:2,2]
    x        = np.moveaxis(x, -1, 0)
    channels = [affine_transform(channel, matrix, offset, output_shape=img_shape[:-1], order=1,
                                 mode='constant', cval=np.average(channel)) for channel in x]
    return np.moveaxis(np.stack(channels, axis=0), 0, -1)

def read_raw_image(p):
    return Image.open(p)

def read_for_validation(x):
    t  = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
    t  = center_transform(t, x.shape)
    x  = transform_img(x, t)
    x -= np.mean(x, keepdims=True)
    x /= np.std(x, keepdims=True) + K.epsilon()
    return x, t

def coord_transform(list, trans):
    result = []
    for x,y in list:
        y,x,_ = trans.dot([y,x,1]).astype(np.int)
        result.append((x,y))
    return result

def read_array(p):
    img = read_raw_image(p).convert('L')
    return img_to_array(img)

def make_bbox(p):
    raw = read_array(p)
    width, height = raw.shape[1], raw.shape[0]
    img,trans         = read_for_validation(raw)
    a                 = np.expand_dims(img, axis=0)
    x0, y0, x1, y1    = model.predict(a).squeeze()
    (u0, v0),(u1, v1) = coord_transform([(x0,y0),(x1,y1)], trans)
    bbox = [max(u0,0), max(v0,0), min(u1,width), min(v1,height)]
    if bbox[0] >= bbox[2] or bbox[1] >= bbox[3]:
        bbox = [0,0,width,height]
    return bbox

In [8]:
bbox_df = pd.DataFrame(columns=['Image','x0','y0','x1','y1']).set_index('Image')

In [9]:
for img in tqdm(data_paths):
    bbox_df.loc[img] = make_bbox(os.path.join(DATA,img))

100%|██████████| 5347/5347 [06:34<00:00, 13.54it/s]


In [10]:
bbox_df.to_csv("bboxs.csv")

In [11]:
(bbox_df.y1).sum()

2607708

In [12]:
bbox_df

,x0,y0,x1,y1
Image,,,,
PM-WWA-20180811-093.jpg,31,109,956,377
PM-WWA-20120430-011.jpg,242,152,1523,609
PM-WWA-20110724-023.jpg,108,172,1198,352
PM-WWA-20160924-452.jpg,67,176,1629,546
PM-WWA-20140703-551.jpg,95,259,2914,880
...,...,...,...,...
PM-WWA-20130423-190.jpg,79,161,943,287
PM-WWA-20070626-014.jpg,80,106,785,206
PM-WWA-20090811-050.jpg,65,58,246,93
